In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from __future__ import absolute_import, division, print_function

import re
import os
import sys
import datetime
import itertools
import warnings

import pandas as pd
import pandas_datareader.data as web
import numpy as np

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs
#from arch import arch_model

import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

# remove warnings
import warnings
warnings.filterwarnings('ignore')

# prevent crazy long pandas prints
pd.options.display.max_columns = 16
pd.options.display.max_rows = 16
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(precision=5, suppress=True)

# Display and Plotting
import matplotlib.pyplot as plt
#set_matplotlib_formats('pdf', 'png')
plt.rcParams['savefig.dpi'] = 80
plt.rcParams['figure.autolayout'] = False
plt.rcParams['figure.figsize'] = 16, 8
plt.rcParams['axes.labelsize'] = 16
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['axes.titleweight'] = 'bold'
plt.rcParams['font.size'] = 16
plt.rcParams['lines.linewidth'] = 2.0
plt.rcParams['lines.markersize'] = 8
plt.rcParams['legend.fontsize'] = 14
plt.rcParams['text.usetex'] = False
#plt.rcParams['font.family'] = "serif"
plt.rcParams['font.serif'] = "cm"
plt.rcParams['text.latex.preamble'] = b"\usepackage{subdepth}, \usepackage{type1cm}"

import seaborn as sns
sns.set_style("whitegrid", {'axes.grid' : False})


from ipywidgets import interactive, widgets, RadioButtons, ToggleButtons, Select, FloatSlider, FloatProgress

from IPython.display import set_matplotlib_formats, Image

# 4. Stationarize a time series

For this section, the function <code>ts_diagnostics</code> is introduced to monitor the effort of stationarizing the time series.

In [ ]:
# load passenger data set and save to DataFrame
pas = pd.read_csv('./data/passengers.csv', header=0, index_col=0, parse_dates=True, sep=';')
y = pas['n_passengers']

In [ ]:
def ts_diagnostics(y, lags=None, title=''):
    '''
    Calculate acf, pacf, qq plot and Augmented Dickey Fuller test for a given time series
    '''
    if not isinstance(y, pd.Series):
        y = pd.Series(y)
        
    # weekly moving averages (5 day window because of workdays)
    rolling_mean = pd.rolling_mean(y, window=12)
    rolling_std = pd.rolling_std(y, window=12)
    
    fig = plt.figure(figsize=(14, 12))
    layout = (3, 2)
    ts_ax = plt.subplot2grid(layout, (0, 0), colspan=2)
    acf_ax = plt.subplot2grid(layout, (1, 0))
    pacf_ax = plt.subplot2grid(layout, (1, 1))
    qq_ax = plt.subplot2grid(layout, (2, 0))
    hist_ax = plt.subplot2grid(layout, (2, 1))
    
    # time series plot
    y.plot(ax=ts_ax)
    rolling_mean.plot(ax=ts_ax, color='crimson');
    rolling_std.plot(ax=ts_ax, color='darkslateblue');
    plt.legend(loc='best')
    ts_ax.set_title(title);
    
    # acf and pacf
    smt.graphics.plot_acf(y, lags=lags, ax=acf_ax, alpha=0.5)
    smt.graphics.plot_pacf(y, lags=lags, ax=pacf_ax, alpha=0.5) 
    
    # qq plot
    sm.qqplot(y, line='s', ax=qq_ax)
    qq_ax.set_title('QQ Plot')
    
    
    # hist plot
    y.plot(ax=hist_ax, kind='hist', bins=25);
    hist_ax.set_title('Histogram');
    plt.tight_layout();
    plt.show()
    
    # perform Augmented Dickey Fuller test
    print('Results of Dickey-Fuller test:')
    dftest = adfuller(y, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['test statistic', 'p-value', '# of lags', '# of observations'])
    for key, value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)
    return 

In [ ]:
ts_diagnostics(y, lags=30, title='International Airline Passengers')

<strong>Inference</strong>
<ul>
<li>Application of the log transformation resulted in less variation of the processes variance</li>
<li>Process is still not stationary, since test statistic is larger than the critical value</li>
</ul>

## 4.1 First difference

The first difference is used to make a time series trend or <strong>mean stationary</strong>. A first difference can be applied applied by

$$ 
y'_t = y_t - Y_{t-1}
$$

In [ ]:
y_diff = np.diff(y)

In [ ]:
ts_diagnostics(y_diff, lags=30, title='International Aliline Passengers log')

<strong>Inference</strong>
<ul>
<li>Improvement for the Dickey Fuller test statistic, but process is still not stationary</li>
<li>Serial correlation has been reduced</li>
<li>Mean appears mean stationary around 0.</li>
</ul>

## 4.2 Log transformation

The log transformation is used to make a time series stationary on the variance.

$$
y^{log}_t = log(y_t)
$$

Note, that the series will not be stationary on the mean since the original data and not the differenced data is used here

In [ ]:
y_log = np.log(y)

In [ ]:
ts_diagnostics(y_log, lags=30, title='International Aliline Passengers log')

<strong>Inference</strong>
<ul>
<li>Application of the log transformation resulted in less variation of the processes variance</li>
<li>Process is still not stationary, since test statistic is larger than the critical value</li>
<li>Strong serial correlation rmains within the series</li>
<li>The errors do not appear do be normally distributed</li>
</ul>

## 4.3 Log transformed first order difference

$$
y'_t = log(y_t) - log(y_{t-1})
$$

In [ ]:
y_log_diff = np.log(y).diff().dropna()

In [ ]:
ts_diagnostics(y_log_diff, lags=30, title='International Airline Passengers log')

<strong>Inference</strong>
<ul>
<li>Application of the first difference seems to have extracted the trend pattern</li>
<li>However, the process is still not stationary, since test statistic is larger than the critical value</li>
<li>The plotted series can now be interpreted as percentages changes per month</li>
</ul>


## 4.4 Log transformed second order difference

Sicne the time series is still not stationary (as indicated by the Dickey-Fuller test) another differencing step can be included.

In [ ]:
y_log_diff2 = np.log(y).diff().diff(12).dropna()

In [ ]:
ts_diagnostics(y_log_diff2, lags=30, title='International Aliline Passengers log')

## 4.5 Interpreting ACF and PACF

Since there are spikes in the ACF and PACF plots which lie outsight the insignifcant zone it can be concluded that the residuals are still not random. This implies that there is an information pattern left in the residuals which can be extracted by <strong>autoregressive</strong> and <strong>moving average</strong> models.

When looking at the PACF and ACF plot of the log transformed second order differenced time series a significant spike in the residuals at lag 12 can be detected. This is an indication for the seasonal component, which as not been extracted from the series completely. Overall, this preliminary result makes sense, since the analyzed data has a monthly frequency that tends to have a seasonality of 12 months.

## 4.6 Exercise

Analyse any of the time series in the stock data set. What transformations are needed to make the series stationary according to the Dickey Fuller test for stationarity?